# Some Models
> To avoid copy-pasta

In [ ]:
# default_exp resnet_models

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


In [ ]:
# exports
from dataclasses import dataclass
from typing import Union

import torch
import torch.nn
import torch.optim
from torch import nn as nn
from torch.nn import functional as F
from torchvision import models

from batchbald_redux.consistent_mc_dropout import (
    BayesianModule,
    ConsistentMCDropout,
    ConsistentMCDropout2d,
)
from batchbald_redux.model_optimizer_factory import ModelOptimizer, ModelOptimizerFactory

In [ ]:
# exports

import torch.nn as nn
from torch import Tensor
from torch.hub import load_state_dict_from_url
from torch.nn import functional as F

model_urls = {
    "resnet18": "https://download.pytorch.org/models/resnet18-5c106cde.pth",
    "resnet34": "https://download.pytorch.org/models/resnet34-333f7ec4.pth",
    "resnet50": "https://download.pytorch.org/models/resnet50-19c8e357.pth",
    "resnet101": "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth",
    "resnet152": "https://download.pytorch.org/models/resnet152-b121ed2d.pth",
    "resnext50_32x4d": "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth",
    "resnext101_32x8d": "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth",
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(
        self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None
    ):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(
        self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None
    ):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class BayesianResNet(BayesianModule):
    def __init__(
        self,
        block,
        layers,
        num_classes=1000,
        zero_init_residual=False,
        groups=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
    ):
        super().__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                "or a 3-element tuple, got {}".format(replace_stride_with_dilation)
            )
        self.groups = groups
        self.base_width = width_per_group

        # This gets reset when cifar_mod=True below
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        # This gets reset when cifar_mod=True
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.fc = nn.Sequential(
            ConsistentMCDropout(),
            nn.Linear(512 * block.expansion, 256 * block.expansion),
            nn.ReLU(),
            ConsistentMCDropout(),
            nn.Linear(256 * block.expansion, 256 * block.expansion),
            nn.Linear(256, num_classes),
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride), norm_layer(planes * block.expansion)
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def deterministic_forward_impl(self, x: Tensor):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)

        return x

    def mc_forward_impl(self, x: Tensor):
        x = self.fc(x)
        x = F.log_softmax(x, dim=-1)
        return x


def _bayesian_resnet(arch, block, layers, cifar_mod, pretrained, progress, **kwargs):
    model = BayesianResNet(block, layers, **kwargs)
    if cifar_mod:
        model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = torch.nn.Identity()

    if pretrained:
        assert not cifar_mod
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict, strict=False)

    return model


def bayesian_resnet18(*, cifar_mod=False, pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _bayesian_resnet("resnet18", BasicBlock, [2, 2, 2, 2], cifar_mod, pretrained, progress, **kwargs)


def bayesian_resnet34(*, cifar_mod=False, pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _bayesian_resnet("resnet34", BasicBlock, [3, 4, 6, 3], cifar_mod, pretrained, progress, **kwargs)

In [ ]:
# exports


@dataclass
class Cifar10BayesianResnetFactory(ModelOptimizerFactory):
    def create_model_optimizer(self) -> ModelOptimizer:
        model = bayesian_resnet18(cifar_mod=True, num_classes=10)
        optimizer = torch.optim.AdamW(model.parameters(), weight_decay=5e-4)
        return ModelOptimizer(model=model, optimizer=optimizer)

## CIFAR-10 40% baseline

In [ ]:
# experiment

from batchbald_redux.datasets import get_dataset
from batchbald_redux.black_box_model_training import train_with_schedule, evaluate

In [ ]:
# experiment 
split_dataset = get_dataset("CIFAR-10", normalize_like_cifar10=False, train_augmentation=True, validation_set_size=5000)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# experiment

from torch.utils import data

kwargs = {"num_workers": 4, "pin_memory": True}

train_loader = data.DataLoader(split_dataset.train * 0.4, batch_size=128, shuffle=True, **kwargs)
validation_loader = data.DataLoader(split_dataset.validation, batch_size=128, shuffle=False, **kwargs)
test_loader = data.DataLoader(split_dataset.test, batch_size=128, shuffle=False, **kwargs)

model_optimizer = Cifar10BayesianResnetFactory().create_model_optimizer()
train_with_schedule(model=model_optimizer.model, optimizer=model_optimizer.optimizer, train_loader=train_loader, validation_loader=validation_loader, patience_schedule=[3,3,3], factor_schedule=[0.1], max_epochs=60, device="cuda", training_samples=1, validation_samples=10, training_log={})

  2%|1         | 1/60 [00:00<?, ?it/s]

[1/141]   1%|           [00:00<?]

[1/40]   2%|2          [00:00<?]

Epoch metrics: {'accuracy': 0.4036, 'crossentropy': 1.7882676845550538}


[1/141]   1%|           [00:00<?]

...

Epoch metrics: {'accuracy': 0.864, 'crossentropy': 0.49511414707899093}
Epoch 35: 0.864 worse than 0.8658, patience: 3/3!


[1/141]   1%|           [00:00<?]

[1/40]   2%|2          [00:00<?]

Epoch metrics: {'accuracy': 0.8628, 'crossentropy': 0.5081136326342821}
Epoch 36: 0.8628 worse than 0.8658, patience: 4/3!


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0.0005
)

In [ ]:
# experiment

evaluate(model=model_optimizer.model, num_samples=20, loader=test_loader, device="cuda")

[1/79]   1%|1          [00:00<?]

{'accuracy': 0.8638, 'crossentropy': 0.5297188091278077}

## CIFAR-100

In [ ]:
# experiment 
split_dataset = get_dataset("CIFAR-100", normalize_like_cifar10=False, train_augmentation=True, validation_set_size=5000)

Files already downloaded and verified


In [ ]:
# experiment

from torch.utils import data

kwargs = {"num_workers": 4, "pin_memory": True}

train_loader = data.DataLoader(split_dataset.train * 0.4, batch_size=128, shuffle=True, **kwargs)
validation_loader = data.DataLoader(split_dataset.validation, batch_size=128, shuffle=False, **kwargs)
test_loader = data.DataLoader(split_dataset.test, batch_size=128, shuffle=False, **kwargs)

model_optimizer = Cifar10BayesianResnetFactory().create_model_optimizer()
train_with_schedule(model=model_optimizer.model, optimizer=model_optimizer.optimizer, train_loader=train_loader, validation_loader=validation_loader, patience_schedule=[3,3,3,3], factor_schedule=[0.1], max_epochs=80, device="cuda", training_samples=1, validation_samples=10, training_log={})

  2%|1         | 1/60 [00:00<?, ?it/s]

[1/141]   1%|           [00:00<?]

[1/40]   2%|2          [00:00<?]

Epoch metrics: {'accuracy': 0.037, 'crossentropy': 4.40652554473877}


[1/141]   1%|           [00:00<?]

...

Epoch metrics: {'accuracy': 0.4238, 'crossentropy': 2.699748579788208}
Epoch 59: 0.4238 worse than 0.4368, patience: 1/3!


[1/141]   1%|           [00:00<?]

[1/40]   2%|2          [00:00<?]

Epoch metrics: {'accuracy': 0.4228, 'crossentropy': 2.831049083137512}
Epoch 60: 0.4228 worse than 0.4368, patience: 2/3!


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.005
    weight_decay: 0.0005
)

In [ ]:
# experiment

evaluate(model=model_optimizer.model, num_samples=20, loader=test_loader, device="cuda")

[1/79]   1%|1          [00:00<?]

{'accuracy': 0.5724, 'crossentropy': 3.0206112657546997}

57.24% is better than Random acquisition in VAAL (Variational Adversarial Active Learning, https://arxiv.org/abs/1904.00370) and also better than the reported CoreSet AL results: 50.5%?

---

"Task-Aware Variational Adversarial Active Learning" (https://arxiv.org/abs/2002.04709) reports much better numbers for Random on ResNet, however: 64%.

They state:

> ResNet18 [13] was used for all task learners and stochastic gradient descent (SGD) was used with momentum 0.9 and weight decay 0.005. Learning rate was 0.1 for the first 160 epochs and then 0.01 for the last 40 epochs.

We leave this as a TODO.